# Implementation and convergence analysis of  PLCCA (aka partially linear CCA)
Hardoon & Shawe-Taylor, 2011

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize, NonlinearConstraint
import scipy
from sklearn.metrics.pairwise import pairwise_kernels, rbf_kernel
import time

In [4]:
from convergence_plcca import *

In [5]:
data = pd.read_csv('ret_subset.csv')

In [6]:
ret = pd.read_csv('ret_subset.csv')

In [7]:
universe_size = 120

In [8]:
ret = np.array(ret.iloc[:252,1:universe_size])

# trading signals

quick built trading signal to obtain a kernel to test our method

In [9]:
def macd(ret, long=8, short=4, signal_span=9):
    """
    calculates the MACD momentum strategy for a single time series
    input: pandas single series
    output: numpy array of signals
    """
    short_signal = ret.ewm(span=short, adjust=False).mean()
    long_signal = ret.ewm(span=long, adjust=False).mean() 
    macd = short_signal - long_signal
    
    pos = np.zeros(len(ret))
    pos = np.where(macd>0,1,-1)
    return pos

def macd_signals(returns,long=26,short=12,signal_span=9):
    """
    function calculating all the macd signals
    input: pandas dataframe of returns
    output: pandas dataframe of signals
    """
    if type(returns) == np.ndarray:
        returns = pd.DataFrame(returns)
    signals = pd.DataFrame()
    for i in range(returns.shape[1]):
        signals['signal_{}'.format(i)] = macd(returns.iloc[:,i],long,short,signal_span)
    signals.index = returns.index
    return signals.shift(1).fillna(0)

# primal dual CCA
exact implementation

In [ ]:
s = macd_signals(ret)
tr_ret = np.array(s) * ret
kernel = gaussian_kernel(tr_ret)

In [ ]:
W,Z, output = scca_deflator(ret.T,kernel,np.arange(0,ret.shape[1],1),0,0,0.1,50,10,10)

In [ ]:
# verbose example of single portfolio
ww,ee,alphaa,betaa,muu,gammaa,corr,ress = scca(np.array(ret).T,kernel,5,1,0,0.1,50,20,20)

# Convergence analysis

In [10]:
# variables to loop over
n_samples = np.array([250,500])
universes = np.array([10,50,100,200,498])
sparsity = np.array([0.1,0.5,1])
overall_while = np.array([500,100,20])
convergence_w_while = np.array([100,10])
convergence_e_while = np.array([100,10])

col = ['assets','sparsity','overall_while','cvg_w','cvg_e','w','e','cor','res','time','time_o','time_w','time_e','loops_o','loops_w','loops_e']
super_output = pd.DataFrame(columns=col)

In [11]:
for n in n_samples:
    for a in universes:
        super_output.to_csv('convergence_large_500.csv')
        # returns
        X = np.array(data.iloc[:n,1:a])
        # kernel
        K = gaussian_kernel(X)
        for b in sparsity:
            for c in overall_while:
                for d in convergence_w_while:
                    for e in convergence_e_while:
                        # get output
                        try:
                            print("currently on sample : {}, assets : {}, sparsity : {}, overall : {}, w cvg : {}, e cvg : {}".format(n,a,b,c,d,e))
                            start = time.time()
                            W,Z, output = scca_deflator(X.T,K,np.arange(0,X.shape[1],1),0,0,b,c,d,e)
                            end = time.time()
                            super_output.loc[len(super_output.index)] = [a,b,c,d,e,output['W'],output['Z'], output['cor'],output['res'],
                                                                        end-start,output['time_o'],output['time_w'], output['time_e'],
                                                                        output['loops_o'],output['loops_w'], output['loops_e']]
                        except (RuntimeError, TypeError, NameError,ValueError,IndexError) :
                            continue

currently on sample : 500, assets : 10, sparsity : 0.1, overall : 500, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 500, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 500, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 500, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 100, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 100, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 100, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 100, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 20, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 10, sparsity : 0.1, overall : 20, w cvg : 100, e cvg : 10
    
c

    
currently on sample : 500, assets : 100, sparsity : 0.1, overall : 20, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 500, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 500, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 500, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 500, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 100, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 100, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 100, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 100, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 100, sparsity : 0.5, overall : 20, w cvg : 100, e cv

    
currently on sample : 500, assets : 498, sparsity : 0.5, overall : 20, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 498, sparsity : 0.5, overall : 20, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 498, sparsity : 0.5, overall : 20, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 500, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 500, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 500, w cvg : 10, e cvg : 100
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 500, w cvg : 10, e cvg : 10
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 100, w cvg : 100, e cvg : 100
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 100, w cvg : 100, e cvg : 10
    
currently on sample : 500, assets : 498, sparsity : 1.0, overall : 100, w cvg : 10, e cvg

In [13]:
super_output

,assets,sparsity,overall_while,cvg_w,cvg_e,w,e,cor,res,time,time_o,time_w,time_e,loops_o,loops_w,loops_e
0,10,0.1,500,100,100,"[[0.3982978680227293, -0.3725666581477327, 0.5...","[[9.692532571488754e-05, -0.014896427999207078...","[0.07750089800988573, 0.9937030161188857, 0.73...","[0.4612495509950572, 0.0031484919405572157, 0....",0.417315,"[0.03500032424926758, 0.01566314697265625, 0.0...","[0.0030570030212402344, 0.01566314697265625, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.001997709274291992...","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]","[5.0, 3.0, 1.0, 10.0, 5.0, 1.0, 23.0, 19.0, 39.0]","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]"
1,10,0.1,500,100,10,"[[0.3982978680227293, -0.3725666581477327, 0.5...","[[9.692532571488754e-05, -0.014896427999207078...","[0.07750089800988573, 0.9937030161188857, 0.73...","[0.4612495509950572, 0.0031484919405572157, 0....",0.551579,"[0.031250953674316406, 0.01562809944152832, 0....","[0.0, 0.0, 0.0, 0.0, 0.003256082534790039, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.002001523971557617...","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]","[5.0, 3.0, 1.0, 10.0, 5.0, 1.0, 23.0, 19.0, 39.0]","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]"
2,10,0.1,500,10,100,"[[0.39829786802272943, -0.3725666581477327, 0....","[[9.692532571488976e-05, -0.014896427999207054...","[0.07750089800988573, 0.9937030161188857, 0.73...","[0.4612495509950572, 0.0031484919405572157, 0....",9.515574,"[0.05826997756958008, 0.0312347412109375, 0.03...","[0.0009982585906982422, 0.0, 0.006002902984619...","[0.0, 0.0, 0.0010025501251220703, 0.0009999275...","[5.0, 3.0, 2.0, 1.0, 3.0, 1.0, 500.0, 1.0, 1.0]","[5.0, 3.0, 1.0, 10.0, 5.0, 11.0, 11.0, 11.0, 1...","[5.0, 3.0, 2.0, 1.0, 3.0, 1.0, 500.0, 1.0, 1.0]"
3,10,0.1,500,10,10,"[[0.39829786802272943, -0.3725666581477327, 0....","[[9.692532571488976e-05, -0.014896427999207054...","[0.07750089800988573, 0.9937030161188857, 0.73...","[0.4612495509950572, 0.0031484919405572157, 0....",11.240445,"[0.06855964660644531, 0.031226396560668945, 0....","[0.0019884109497070312, 0.0, 0.015622615814208...","[0.0, 0.0, 0.0009982585906982422, 0.0, 0.0, 0....","[5.0, 3.0, 2.0, 1.0, 3.0, 1.0, 500.0, 1.0, 1.0]","[5.0, 3.0, 1.0, 10.0, 5.0, 11.0, 11.0, 11.0, 1...","[5.0, 3.0, 2.0, 1.0, 3.0, 1.0, 500.0, 1.0, 1.0]"
4,10,0.1,100,100,100,"[[0.3982978680227293, -0.3725666581477327, 0.5...","[[9.692532571488754e-05, -0.014896427999207078...","[0.07750089800988573, 0.9937030161188857, 0.73...","[0.4612495509950572, 0.0031484919405572157, 0....",0.595844,"[0.04688715934753418, 0.031246662139892578, 0....","[0.0, 0.0, 0.0, 0.0, 0.015624761581420898, 0.0...","[0.0, 0.0, 0.0006916522979736328, 0.0, 0.0, 0....","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]","[5.0, 3.0, 1.0, 10.0, 5.0, 1.0, 23.0, 19.0, 39.0]","[5.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 1.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,498,1.0,100,10,10,"[[0.0036517453096029744, -0.026985099720592695...","[[0.0014638291525312821, 0.0014231218440732636...","[0.4457491051491966, 0.4528464635926626, 0.792...","[0.2771254474254018, 0.2735767682036688, 0.103...",120.936153,"[0.5212898254394531, 1.7542457580566406, 0.179...","[0.2504842281341553, 0.15400123596191406, 0.05...","[0.0007028579711914062, 0.11419677734375, 0.00...","[17.0, 100.0, 6.0, 7.0, 5.0, 6.0, 6.0, 7.0, 5....","[11.0, 1.0, 5.0, 6.0, 3.0, 2.0, 2.0, 7.0, 8.0,...","[17.0, 100.0, 6.0, 7.0, 5.0, 6.0, 6.0, 7.0, 5...."
176,498,1.0,20,100,100,"[[0.0036512640732993414, -0.02698583164300156,...","[[0.0014638291525312412, 0.0014231218440732363...","[0.44574911956246294, 0.45284106471866176, 0.7...","[0.2771254402187685, 0.27357946764066904, 0.10...",104.881486,"[0.4273216724395752, 0.3700089454650879, 0.160...","[0.2596557140350342, 0.05168771743774414, 0.06...","[0.0, 0.024183273315429688, 0.0049889087677001...","[11.0, 20.0, 6.0, 7.0, 5.0, 6.0, 6.0, 7.0, 5.0...","[25.0, 1.0, 5.0, 6.0, 3.0, 2.0, 2.0, 7.0, 8.0,...","[11.0, 20.0, 6.0, 7.0, 5.0, 6.0, 6.0, 7.0, 5.0..."
177,4